In [1]:
import torch
import numpy as np
import math
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import torch.nn as nn
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import r2_score
import random
import matplotlib as mpl
import os
import gc
import pandas as pd
import csv
from numpy import *
from torch.utils.tensorboard import SummaryWriter
from datetime import date
import json
# mpl.rcParams['figure.dpi'] = 180

In [2]:
# # different amino acids
amino_acid = ['A', 'V', 'F', 'I', 'L','D','E','K','S','T','Y','C','N','Q', 'P','M', 'R', 'H', 'W', 'G','X'] # X is the undetermined amino acid, so total length is 21
print('Number of unique amino acids are', np.shape(np.unique(amino_acid))[0])

def onehotseq(sequence):
  seq_len = np.shape(sequence)[0]
  # print('Seq len is', seq_len)
  seq_en = np.zeros(( seq_len, np.shape(amino_acid)[0]))
  for i in range(seq_len):
    if sequence[i] in amino_acid:
      pos = amino_acid.index(sequence[i])
      seq_en[i,pos] = 1
    else:
      pos = amino_acid.index('X')
      seq_en[i,pos] = 1

  return seq_en

Number of unique amino acids are 21


In [3]:
### input sequence data
def one_hot_encoding(input_sequence, num_files, seq_length):
    ohe = np.zeros((input_sequence.shape[0], input_sequence.shape[1],input_sequence.shape[2], len(amino_acid)))

    for i in range(input_sequence.shape[0]):
        for j in range(input_sequence.shape[1]):
            seq_len = int(seq_length[i,j])
            if seq_len > 0:
                seq_en = onehotseq(input_sequence[i,j,0:seq_len])
                ohe[i,j,0:seq_len,:] = seq_en
    return ohe

In [4]:
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')

# LSTM Model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_layers, seq_len,num_classes=1):
        super(RNN, self).__init__()
        self.num_layers = num_layers
        self.hidden_size1 = hidden_size1
        self.hidden_size2 = hidden_size2
        self.seq_len = seq_len

        self.bnn1 = nn.Linear(input_size, 32)
        self.bnn2 = nn.Linear(32,64)
        self.bnn3 = nn.Linear(64,64)
        self.bnn4 = nn.Linear(64,64)

        self.lstm1 = nn.LSTM(64, hidden_size1, num_layers, batch_first=True, bidirectional=True, dropout=0.5)
        self.bn_lstm1 = nn.BatchNorm1d(2*hidden_size1,device=device)  
        # self.lstm2 = nn.LSTM(2*hidden_size1, hidden_size2, num_layers, batch_first=True, bidirectional=True, dropout=0.5)
        # self.bn_lstm2 = nn.BatchNorm1d(2*hidden_size2,device=device)
        self.nn1 = nn.Linear(2*hidden_size1, 2*hidden_size1)
        self.nn2 = nn.Linear(2*hidden_size1, 512)
        self.nn3 = nn.Linear(512, 512)
        self.nn4 = nn.Linear(512, 256)
        self.nn5 = nn.Linear(256, 256)
        self.nn6 = nn.Linear(256, 128)
        self.nn7 = nn.Linear(128, 32)
        self.nn8 = nn.Linear(32, 1)

    
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        # self.batch = nn.BatchNorm1d()
        self.drop = nn.Dropout(p=0.5)


        
    def forward(self, x, array_lengths):
        # Set initial hidden states (and cell states for LSTM)
        # print(x.size(0))
        inital_seq_len = x.size(1)
        x = Variable(x.float()).to(device)

        x = torch.reshape(x, (x.size(0)*x.size(1), x.size(2)))

        ## before nn
        out = self.bnn1(x)
        out = self.relu(out)
        out = self.bnn2(out)
        out = self.relu(out)
        out = self.bnn3(out)
        out = self.relu(out)
        out = self.bnn4(out)
        out = self.relu(out)

        ## reshaping again
        out = torch.reshape(out, (-1, inital_seq_len, out.size(1)))
        # print(out.size())
        # print(aaaaa)

        # out = torch.permute(out, (0,2,1))
        
        pack = nn.utils.rnn.pack_padded_sequence(out, array_lengths, batch_first=True, enforce_sorted=False)
        h0 = Variable(torch.zeros(2*self.num_layers, x.size(0), self.hidden_size1).to(device))
        c0 = Variable(torch.zeros(2*self.num_layers, x.size(0), self.hidden_size1).to(device))
        h1 = Variable(torch.zeros(2*self.num_layers, self.hidden_size1, self.hidden_size2).to(device))
        c1 = Variable(torch.zeros(2*self.num_layers, self.hidden_size1, self.hidden_size2).to(device))
        
        # Forward propagate RNN
        out, _ = self.lstm1(pack, (h0,c0))
        del(h0)
        del(c0)
        # out, _ = self.lstm2(out, (h1,c1))
        gc.collect()
        unpacked, unpacked_len = torch.nn.utils.rnn.pad_packed_sequence(out, batch_first=True)
        this_batch_len = unpacked.size(1)
        out = unpacked
        # print('before', out.size())
        out = torch.reshape(out, (out.size(0)*out.size(1), out.size(2)))

        ##nn
        out = self.nn1(out)
        out = self.relu(out)
        out = self.nn2(out)
        out = self.relu(out)
        out = self.nn3(out)
        out = self.relu(out)
        out = self.nn4(out)
        out = self.relu(out)
        out = self.nn5(out)
        out = self.relu(out)
        out = self.nn6(out)
        out = self.relu(out)
        out = self.nn7(out)
        out = self.relu(out)
        out = self.nn8(out)
        
        ## reshaping
        out = torch.reshape(out, (-1, this_batch_len, 1))
        # print(out.size()) 
        # print(aaaaa)   
        

        return out

model_test = torch.load('../b_factor_model.pth', map_location='cuda:2')
model_test.eval().to(device)

RNN(
  (bnn1): Linear(in_features=21, out_features=32, bias=True)
  (bnn2): Linear(in_features=32, out_features=64, bias=True)
  (bnn3): Linear(in_features=64, out_features=64, bias=True)
  (bnn4): Linear(in_features=64, out_features=64, bias=True)
  (lstm1): LSTM(64, 512, batch_first=True, dropout=0.5, bidirectional=True)
  (bn_lstm1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (nn1): Linear(in_features=1024, out_features=1024, bias=True)
  (nn2): Linear(in_features=1024, out_features=512, bias=True)
  (nn3): Linear(in_features=512, out_features=512, bias=True)
  (nn4): Linear(in_features=512, out_features=256, bias=True)
  (nn5): Linear(in_features=256, out_features=256, bias=True)
  (nn6): Linear(in_features=256, out_features=128, bias=True)
  (nn7): Linear(in_features=128, out_features=32, bias=True)
  (nn8): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
  (tanh): Tanh()
  (drop): Dropout(p=0.5, inplace=False)
)

In [5]:
def b_factor_cal(ohe, num_files, seq_length):
  
  b_factor = np.zeros((ohe.shape[0], ohe.shape[1], ohe.shape[2]))
  ohe = torch.from_numpy(ohe).to(device)
  # max_seq_len = int(np.max(seq_length))
  # print(max_len)
  with torch.no_grad():
    for i in range(ohe.shape[0]):
      if int(num_files[i,]) !=0:
        input_x = ohe[i,0:int(num_files[i,]),:,:]        
        input_x = torch.reshape(input_x, (input_x.shape[0],int(np.max(seq_length)),len(amino_acid)))  
        # print(input_x.shape)
        array_lengths = torch.from_numpy(seq_length[i,0:int(num_files[i,])])
        outputs = model_test(input_x, array_lengths)
        outputs = torch.reshape(outputs, (outputs.shape[0], outputs.shape[1]))
        b_factor[i,0:int(num_files[i,]),0:int(torch.max(array_lengths))] = outputs.to('cpu').numpy()
  
  return b_factor

# ohe = ohe.to('cpu').numpy()

Making heat map for RC 3

In [6]:
def rc_cal(b_factor, ohe, input_sequence, num_files, seq_length):

    filter = 7 ##THESE ARE VARIABLES. YOU CAN CHANGE THESE TO SUIT YOUR REQUIREMENTS. THIS DEFINES THE MAX(M) VALUE IN THE PAPER. MAX(M)=FILTER-1
    stride = 1
    rc = filter-1
    heat_map = np.zeros((input_sequence.shape[0], len(amino_acid), len(amino_acid), rc, 2))


    for prot in range(heat_map.shape[0]):
        for fl in range(int(num_files[prot,])):
            # print(num_files[prot,])
            
            subs_map = np.zeros((len(amino_acid), len(amino_acid), rc,2))
            index = 0
            conti = True 
            b_mean = np.mean(b_factor[prot,fl,0:int(seq_length[prot,fl])])
            b_std =  np.std(b_factor[prot,fl,0:int(seq_length[prot,fl])])        
                
            while conti:
                conv_unit = np.argmax(ohe[prot,fl,index:index+filter, :], axis=1)
                b_unit = b_factor[prot,fl,index:index+filter]
                look = np.arange(1,len(b_unit))
                
                for i in look:
                    subs_map[conv_unit[0], conv_unit[i],i-1,0] += (( b_unit[0] - b_mean)/b_std)
                    subs_map[conv_unit[0], conv_unit[i],i-1,1] += (( b_unit[i] - b_mean)/b_std)
                    
                index += stride
                conti = ((index + filter) < seq_length[prot,fl])
            
            for m in range(rc):
                heat_map[prot,:,:,m,:] += (subs_map[:,:,m,:]/(seq_length[prot,fl]-(m+1)))
        
        if num_files[prot,] != 0:   
            heat_map[prot,:,:,:,:] =  heat_map[prot,:,:,:,:]/num_files[prot,]

    heat_map = heat_map[:,0:len(amino_acid)-1,0:len(amino_acid)-1,:,:]       
    
    return heat_map

In [7]:
def corr_cals(heat_map, output_properties, id, num_files):
    corr = np.zeros((heat_map.shape[1], heat_map.shape[2], heat_map.shape[3], heat_map.shape[4]))

    for i in range(corr.shape[0]):
        for j in range(corr.shape[1]):
            for k in range(corr.shape[2]):
                for l in range(corr.shape[3]):
                    index = ((np.isnan(output_properties[:,id])==False) & (num_files != 0))
                    pcc_prop = output_properties[index,id].reshape((1,-1))
                    pcc_fea = heat_map[index,i,j,k,l].reshape((1,-1))
                    if np.isnan(np.corrcoef(pcc_prop,pcc_fea)[0,1]):
                        corr[i,j,k,l] = 0
                    else:
                        corr[i,j,k,l] = np.corrcoef(pcc_prop,pcc_fea)[0,1]
    
    return corr

In [8]:
def prop_cal(present, prop):
    output_properties = np.zeros((len(present), 12))
    count = 0
    for ele in present:
        pos = int(np.where(prop[:,0]==ele)[0])
        output_properties[count,:] = prop[pos,1:13]
        count += 1
    return output_properties

In [9]:
def feature_eng_cals(heat_map,feature_name, current):

    good_features = 10000
    # good_features = np.sum(corr>lim) + np.sum(corr<-lim)
    feature_engineered = np.zeros((heat_map.shape[0], good_features))

    count = 0
    
    for i in range(heat_map.shape[1]):
        for j in range(heat_map.shape[2]):
            for k in range(heat_map.shape[3]):
                for l in range(heat_map.shape[4]):
                    name = current+ '_' +str(amino_acid[i])+ '_'  + str(amino_acid[j]) + '_'+ str(k+1) + '_'+str(l+1)
                    if name in feature_name:
                        feature_engineered[:,count] = heat_map[:,i,j,k,l]
                        count += 1
    
    print(f'Number of features in spidroin {current} are', count)            
   
    return feature_engineered[:,0:count],feature_name[0:count, ]

In [10]:
accepted_type = ['MaSp1', 'MaSp2', 'MaSp3', 'MaSp','MiSp', 'Spidroin']
feature_names = np.load('./young_features_name.npy', allow_pickle=True)
user_input = {'MaSp1': 'QSPVLGLVQQFSLGSKPDVSISTLCPEEPALKSTEPNDSFEYDVEFIPVFDSAANFVPVLPEIEEEIDLIDAEENAEEVVYRRTAPSFNLEDFENIFLLTLKDGGERIKFLTKALLKTKGQHSHNSLKLGHFVKKLSEVFGSMRYNNEYLTYSELYVQLLTETLIAALEIIRETDSSCLTVPSAPSELPQYVNALYNILI',
              'MaSp2': None,
              'MaSp3':None,
              'MaSp':None,
              'MiSp':None,
              'Spidroin':None}
all_features =  np.zeros((1, 10000))
count = 0

for i in range(len(accepted_type)):
    spidroin = [accepted_type[i]]
    seq = user_input[accepted_type[i]]
    if seq is not None:
        seq_len = len(list(seq))
        input_sequence = np.zeros((1, 1, seq_len), dtype=object)
        input_sequence[0,0,:] = list(seq)
        num_files = np.ones((1,))
        seq_length = np.ones((1,1))*seq_len
        ohe = one_hot_encoding(input_sequence, num_files, seq_length)
        b_factor = b_factor_cal(ohe, num_files, seq_length)
        heat_map = rc_cal(b_factor, ohe, input_sequence, num_files, seq_length)
        features, names  = feature_eng_cals(heat_map, feature_names,  accepted_type[i])
        size = features.shape[1]
        all_features[:,count:count+size] = features
        count += size
    else:
        heat_map = np.zeros((1,len(amino_acid), len(amino_acid), 6, 2))
        features, names  = feature_eng_cals(heat_map, feature_names,  accepted_type[i])
        size = features.shape[1]
        all_features[:,count:count+size] = features
        count += size
        

all_features =  all_features[:,0:count]
print('Shape of all_features', all_features.shape)

Number of features in spidroin MaSp1 are 132
Number of features in spidroin MaSp2 are 77
Number of features in spidroin MaSp3 are 321
Number of features in spidroin MaSp are 63
Number of features in spidroin MiSp are 26
Number of features in spidroin Spidroin are 206
Shape of all_features (1, 825)


**Prediction of property**

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# CNN + RNN Model
class network(nn.Module):
    def __init__(self):
        super(network, self).__init__()
        
        # nn layers
        
        self.nn = nn.Sequential(nn.Linear(825,128),
                                nn.ReLU(),
                                nn.Linear(128,64),
                                nn.ReLU(),
                                nn.Linear(64,24),
                                nn.ReLU(),
                                nn.Linear(24,12),
                                nn.ReLU(),
                                nn.Linear(12,8),
                                nn.ReLU(),
                                nn.Linear(8,1)
                                )
    

        
    def forward(self, x):
        # out3 = x3
        # out5 = x5
        out = self.nn(x)

        return out

model = torch.load('./model/best.pth', map_location='cuda')
model.eval().to(device)

network(
  (nn): Sequential(
    (0): Linear(in_features=825, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=24, bias=True)
    (5): ReLU()
    (6): Linear(in_features=24, out_features=12, bias=True)
    (7): ReLU()
    (8): Linear(in_features=12, out_features=8, bias=True)
    (9): ReLU()
    (10): Linear(in_features=8, out_features=1, bias=True)
  )
)

In [12]:
all_features = torch.from_numpy(all_features).to(device).type(dtype=torch.float32)

with torch.no_grad():
    predicted_prop = model(all_features)
    print('Predicted Youngs modulus', round(predicted_prop.item(),2))

Predicted Youngs modulus 9.59
